In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .master("local[*]") \
  .appName("test") \
  .config("spark.executor.instances", "8") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/14 21:06:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
df_green = spark.read.parquet("../../../data/pg/green/2020/01/part-00000-fc72b89e-f8fc-4908-9c19-50a5c59c6a30-c000.snappy.parquet")

In [14]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
  .select(columns) \
  .rdd
  


In [15]:
import pandas as pd 

In [16]:
rows = duration_rdd.take(10)

In [17]:
rows

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 18, 30, 27), PULocationID=10, DOLocationID=215, trip_distance=2.18),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 14, 36), PULocationID=205, DOLocationID=264, trip_distance=12.52),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 12, 55, 57), PULocationID=75, DOLocationID=238, trip_distance=1.51),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 13, 54, 57), PULocationID=7, DOLocationID=82, trip_distance=3.22),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 22, 29, 42), PULocationID=25, DOLocationID=87, trip_distance=2.7),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 8, 9, 4), PULocationID=131, DOLocationID=92, trip_distance=7.85),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 17, 46, 47), PULocationID=74, DOLocationID=74, trip_distance=0.86),
 Row(VendorID=2, lpep_pickup_datetime=datetim

In [18]:
pd.DataFrame(rows, columns=columns)

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2.0,2020-01-03 18:30:27,10,215,2.18
1,NaN,2020-01-05 14:36:00,205,264,12.52
2,2.0,2020-01-14 12:55:57,75,238,1.51
3,2.0,2020-01-29 13:54:57,7,82,3.22
4,1.0,2020-01-24 22:29:42,25,87,2.70
5,NaN,2020-01-08 09:04:00,131,92,7.85
6,2.0,2020-01-03 17:46:47,74,74,0.86
7,2.0,2020-01-18 15:19:16,97,181,0.08
8,2.0,2020-01-19 09:57:59,41,74,0.74
9,2.0,2020-01-11 12:08:30,260,82,2.90


In [19]:
df = pd.DataFrame(rows, columns=columns)
list(df.itertuples())

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-03 18:30:27'), PULocationID=10, DOLocationID=215, trip_distance=2.18),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-05 14:36:00'), PULocationID=205, DOLocationID=264, trip_distance=12.52),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-14 12:55:57'), PULocationID=75, DOLocationID=238, trip_distance=1.51),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 13:54:57'), PULocationID=7, DOLocationID=82, trip_distance=3.22),
 Pandas(Index=4, VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-24 22:29:42'), PULocationID=25, DOLocationID=87, trip_distance=2.7),
 Pandas(Index=5, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-08 09:04:00'), PULocationID=131, DOLocationID=92, trip_distance=7.85),
 Pandas(Index=6, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-03 17:46:47'), PULocationID=74, DOLocationID=74, trip_distance=0.86),
 Pandas(I

In [20]:
def infinite_seq():
  i = 0
  while True:
    yield i
    i = i + 1
    
    if i > 15:
      break
    
seq = infinite_seq()
list(seq)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [21]:
for i in seq:
  print(i)
  if i >= 10:
    break

In [22]:
# model = ...

def model_predict(df):
  # y_pred = model.predict(df)
  y_pred = df.trip_distance * 5
  return y_pred

In [23]:
def apply_model_in_batch(rows):
  df = pd.DataFrame(rows, columns=columns)
  predictions = model_predict(df)
  df['predicted_duration']  = predictions

  for row in df.itertuples():
    yield row  

In [24]:
duration_rdd.mapPartitions(apply_model_in_batch).take(10)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-03 18:30:27'), PULocationID=10, DOLocationID=215, trip_distance=2.18, predicted_duration=10.9),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-05 14:36:00'), PULocationID=205, DOLocationID=264, trip_distance=12.52, predicted_duration=62.599999999999994),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-14 12:55:57'), PULocationID=75, DOLocationID=238, trip_distance=1.51, predicted_duration=7.55),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 13:54:57'), PULocationID=7, DOLocationID=82, trip_distance=3.22, predicted_duration=16.1),
 Pandas(Index=4, VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-24 22:29:42'), PULocationID=25, DOLocationID=87, trip_distance=2.7, predicted_duration=13.5),
 Pandas(Index=5, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-08 09:04:00'), PULocationID=131, DOLocationID=92, trip_distance=7.85, predicted_du

In [25]:
df_predicts = duration_rdd \
  .mapPartitions(apply_model_in_batch) \
  .toDF() \
  .drop('Index')

In [26]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              10.9|
|62.599999999999994|
|              7.55|
|              16.1|
|              13.5|
|             39.25|
|               4.3|
|               0.4|
|               3.7|
|              14.5|
|              17.2|
|              15.9|
|             47.35|
|              4.25|
|              13.5|
|57.300000000000004|
|             17.95|
|               4.0|
|               5.0|
|               8.5|
+------------------+
only showing top 20 rows

